In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19416971
paper_name = 'khozoie_avery_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/jbc.M109.005843-1.xls', sheet_name='Initial QN Screen', skiprows=3)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5664 x 13


In [7]:
original_data.head()

,ORF,Plate,Row,Col,Control A,Control B,Quinine A,Quinine B,MEAN Control,MEAN +Quinine,Growth Ratio (GR),Plate Median,Adjusted GR
0,EMPTY,1,A,1,0.117,0.116,0.110,0.110,0.1165,0.1100,SLOW,1.56586,SLOW
1,YHL047C,1,A,2,0.471,0.443,0.489,0.466,0.4570,0.4775,0.957068,1.56586,0.611209
2,YHL046C,1,A,3,0.736,0.509,0.406,0.454,0.6225,0.4300,1.44767,1.56586,0.924523
3,YHL045W,1,A,4,0.667,0.461,0.509,0.485,0.5640,0.4970,1.13481,1.56586,0.724719
4,YHL044W,1,A,5,0.117,0.119,0.111,0.110,0.1180,0.1105,SLOW,1.56586,SLOW


In [8]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [10]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [11]:
to_drop = original_data.loc[original_data['ORF']=='EMPTY',]

In [12]:
original_data.drop(index=to_drop.index, inplace=True)

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

                  ORF  Plate Row Col  Control A  Control B  Quinine A  \
index_input                                                             
5604         WTBY4743     71   D   1      0.082      0.081      0.082   
5605         WTBY4743     71   D   2      0.083      0.081      0.084   

             Quinine B  MEAN Control  MEAN +Quinine Growth Ratio (GR)  \
index_input                                                             
5604             0.083        0.0815         0.0825              SLOW   
5605             0.084        0.0820         0.0840              SLOW   

             Plate Median Adjusted GR  
index_input                            
5604                  NaN        SLOW  
5605                  NaN        SLOW  


In [14]:
original_data = original_data.loc[t,:]

In [15]:
# Eliminate the slow growin strains for which no accurate growth ratio could be calculated
to_drop = original_data.loc[original_data['Adjusted GR']=='SLOW',]

In [16]:
original_data.drop(index=to_drop.index, inplace=True)

In [17]:
original_data.shape

(4394, 13)

In [18]:
original_data['Growth Ratio (GR)'] = pd.to_numeric(original_data['Growth Ratio (GR)'], errors='coerce')

In [19]:
# Reverse the growth ratio so that lower values correspond to decreased growth and viceversa (originally, GR is reported as untreated vs treated)
original_data['GR2'] = 1 / original_data['Growth Ratio (GR)']

In [20]:
# Normalize by plate median (as done oridinally)
def normalize_by_plate_median(plate_data):
    plate_median = plate_data['GR2'].median()
    plate_data['GR2_adjusted'] = plate_data['GR2'] / plate_median
    return plate_data

In [21]:
original_data2 = original_data.groupby('Plate').apply(normalize_by_plate_median)

In [22]:
original_data2.head()

,ORF,Plate,Row,Col,Control A,Control B,Quinine A,Quinine B,MEAN Control,MEAN +Quinine,Growth Ratio (GR),Plate Median,Adjusted GR,GR2,GR2_adjusted
index_input,,,,,,,,,,,,,,,
1,YHL047C,1,A,2,0.471,0.443,0.489,0.466,0.4570,0.4775,0.957068,1.56586,0.611209,1.044858,1.636101
2,YHL046C,1,A,3,0.736,0.509,0.406,0.454,0.6225,0.4300,1.447674,1.56586,0.924523,0.690763,1.081638
3,YHL045W,1,A,4,0.667,0.461,0.509,0.485,0.5640,0.4970,1.134809,1.56586,0.724719,0.881206,1.379845
5,YHL043W,1,A,6,0.659,0.576,0.495,0.497,0.6175,0.4960,1.244960,1.56586,0.795064,0.803239,1.257760
7,YHL041W,1,A,8,0.617,0.428,0.564,0.461,0.5225,0.5125,1.019512,1.56586,0.651088,0.980861,1.535892


In [23]:
original_data2.set_index('ORF', inplace=True)
original_data2.index.name='orf'

In [24]:
original_data2['data'] = original_data2['GR2_adjusted']

In [25]:
original_data2 = original_data2[['data']].copy()

In [26]:
original_data2 = original_data2.groupby(original_data2.index).mean()

In [27]:
original_data2.head()

,data
orf,
YAL002W,1.270912
YAL004W,0.839775
YAL005C,1.331802
YAL007C,0.730275
YAL008W,0.736262


In [28]:
original_data2.shape

(4251, 1)

# Prepare the final dataset

In [29]:
data = original_data2.copy()

In [30]:
dataset_ids = [16533]
datasets = datasets.reindex(index=dataset_ids)

In [31]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [32]:
data.head()

dataset_id,16533
data_type,value
orf,
YAL002W,1.270912
YAL004W,0.839775
YAL005C,1.331802
YAL007C,0.730275
YAL008W,0.736262


## Subset to the genes currently in SGD

In [33]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [34]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16533
,data_type,value
gene_id,orf,
2,YAL002W,1.270912
1863,YAL004W,0.839775
4,YAL005C,1.331802
5,YAL007C,0.730275
6,YAL008W,0.736262


# Normalize

In [35]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [36]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [37]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          16533          
data_type           value    valuez
gene_id orf                        
2       YAL002W  1.270912  1.452056
1863    YAL004W  0.839775 -0.653321
4       YAL005C  1.331802  1.749400
5       YAL007C  0.730275 -1.188045
6       YAL008W  0.736262 -1.158808

# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [39]:
from IO.save_data_to_db3 import *

In [40]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 19416971...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.83s/it]

Updating the data_modified_on field...
